In [ ]:
from tensorflow as tf
import numpy as np
from tensorflow.python.framework import dtypes 
from tensorflow.python.framework.ops import convert_to_tensor


img_mean = tf.constant([123.68, 116.779, 103.939], dtype = tf.float32)

class DataGeneraotr(object):
    def __init__(self, textfile, num_classes, batch_size):
        self.textfile = textfile
        self.num_classes = num_classes
        self.batch_size = batch_size
        self._read_text_file()
        self.num_samples = len(self.label)
        self.img_batch, self.label_batch = self._image_decode()
        
        
    def _read_text_file(self):
        self.imgpath = []
        self.label = []
        with open(self.textfile, "r") as f:
            lines = f.readlines()
            for data in lines:
                # the rule for spliting depends on how data are stored
                imgpath, label = lines.split(" ")
                self.imgpath.append(imgpath)
                self.label.append(label)
            
    def _image_decode(self):
        # connvert imgpath and label into tensor
        img = tf.convert_to_tensor(self.imgpath, dtype = dtypes.float32)
        label = tf.convert_to_tensor(self.label, dtype = dtypes.int32)
        # create input queues
        file_queue = tf.train.slice_input_producer([self.imgpath, self.labelfile], shuffle = Flase)
        # process a file in queue into a image and label
        file_content = tf.read_file(file_queue[0])
        # decode_*** depends the type of img(png, jpg...), channel depends on RGB or ..
        img_data = tf.image.decode_png(file_content, channel = 3)
        img = tf.image.resize(img_data, shape = [224, 224])
        img_normal = tf.subtract(img, img_mean)
        # convert RGB to BGR
        img_bgr = img_normal[:, :, ::-1]
        # get one-hot label
        label = tf.one_hot(file_queue[1], self.num_classes, dtype = tf.unit8)
        # shuffle the data and label, capacity means an maximum number of data, depends on the 
        # min_after_dequeue mean minimum number of data after dequeue
        img_batch, label_batch = tf.train.shuffle_batch([img_bgr, label], batch_size = self.batch_size,
                                                       capacity = self.num_sampels, min_after_dequeue = 1000)
        return img_batch, label_batch
    